In [1]:
#!/usr/bin/env python
import os
from resgds import *
import bragg 
import gdspy # gds library
import numpy as np
from subprocess import call # Use to call kaloput_viewer bash script
import psutil # Use to check if klayout is running already···

# Layout filename
layout_file = 'test.gds'

# Parameters
sub_x = 10000
sub_y = 10000
wc = 200 #8.11  Length of cavity
gc = 200 #17.85  Gap b/w conductor and substrate
lc = 8108.45 # Conductor width of cavity

wlow = 300  #30.44 Width of low impedance section
glow = .5*(wc + 2*gc - wlow) #6.685 Gap of low impedance section
llow = 4051.32

whigh = 50 #2
ghigh = .5*(wlow + 2*glow - whigh)
lhigh = 4051.32

rlow = 50 # Radius of low impedance section
rhigh = 50

# Setup gds cell and gds object
poly_cell = gdspy.Cell('POLYGONS')
rs = Shapes(poly_cell)

# Substrate [layer 0]
sub = BuildRect(poly_cell, sub_x, sub_y, layer=0)
sub.make(0,0)

# Make Bragg Mirror sections
x0 = 0
y0 = sub_y/2

#layer 2 - Feed line 10, .5, 300, 250, 1500, 3000, 2
bond_pad = 200
central_conductor = 50
ratio = 0.5
H = 1000
W = 4000
r = 500

d_dots = 40

layout = LayoutComponents(poly_cell,sub_x, sub_y,layer=1)
layout.make_antidot_array(0,0,10,d_dots,1)
dots = layout.antidot_array(0,0,10,d_dots,1)

layout2 = LayoutComponents(poly_cell,sub_x, sub_y,layer=2)
feedline = layout2.make_feedline(central_conductor, ratio, r, W, H, bond_pad, d_dots)

# Check if klayout is already running. If not, write gds and open klayout.·
# If it is, just update the gds file
if("klayout" in (p.name() for p in psutil.process_iter())):
    #Write the pattern as a gds file
    gdspy.write_gds(layout_file, unit=1.0e-6, precision=1.0e-9)
else:
    gdspy.write_gds(layout_file, unit=1.0e-6, precision=1.0e-9)
    kl = call('./klayout_viewer %s' %layout_file,shell=True)